<a href="https://colab.research.google.com/github/Manish533005/nlp_chatbot/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [379]:
import pickle
import numpy as np
from keras.models import load_model
import json
import random
from fuzzywuzzy import process
import spacy
model =load_model("chatbot_6.h5")
intents = json.loads(open('chatbot_dataset.json').read())
intents_1=json.loads(open("food_document.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
nlp=spacy.load("en_core_web_sm")
print(len(words))
print(len(classes))
items_append=[]
price_append=[]
rem_value=0

79
14


In [355]:
!pip install fuzzywuzzy

In [356]:
print(intents_1)

{'intents': [{'tag': 'Non-Veg Pizza', 'items': ['chicken pepperoni', 'Pepper Barbecue', 'Dominator', 'chicken Sausage', 'pepper barbecue and onion', 'Indi chicken tikka', 'Moroccan Spice Pasta Pizza'], 'size': ['small', 'medium', 'large'], 'price': [350]}, {'tag': 'Veg Pizza', 'items': ['peppy paneer', 'Mexican green wave', 'Veggie paradise', 'veg extravaganza', 'deluxe veggie', 'paneer Makhani', 'creamy tomato pasta '], 'size': ['small', 'medium', 'large'], 'price': [300]}, {'tag': 'Beverages', 'items': ['pepsi', 'slice', '7up', 'mirinda', 'nimbooz'], 'size': [], 'price': [50]}, {'tag': 'Slides', 'items': ['garlic breadsticks', 'taco mexicana veg', 'Butterscotch mouse cake'], 'size': [], 'price': [99]}]}


In [357]:
print(words)

['-', '-PRON-', '7up', 'add', 'available', 'barbecue', 'beverage', 'breadstick', 'butterscotch', 'bye', 'cake', 'cancel', 'cancle', 'change', 'chat', 'chicken', 'combo', 'confirm', 'cool', 'creamy', 'deluxe', 'dominator', 'drink', 'edit', 'extravaganza', 'food', 'garlic', 'goodbye', 'green', 'hard', 'hello', 'hey', 'hi', 'hungry', 'indi', 'item', 'large', 'later', 'makhani', 'medium', 'menu', 'mexican', 'mexicana', 'mirinda', 'moroccan', 'mouse', 'nice', 'nimbooz', 'non', 'nonveg', 'onion', 'order', 'paneer', 'paradise', 'pasta', 'pepper', 'pepperoni', 'peppy', 'pepsi', 'pizza', 'remove', 'request', 'sausage', 'slice', 'slide', 'small', 'spice', 'taco', 'tikka', 'till', 'time', 'today', 'tomato', 'variety', 'veg', 'veggie', 'view', 'want', 'wave']


In [358]:
all_stopwords = nlp.Defaults.stop_words
sentence_2=[]
sent=[]

In [359]:
#tokenize each word
def fuzzy(sentence):
    str2match=sentence
    strOptions=words
    Ratios=process.extract(str2match,strOptions)
    highest=process.extractOne(str2match,strOptions)
    print(highest[0])
    print("above is fuzzy wuzzy")
    return highest[0]
def clean_up_sentence(sentence):
        sentence_2=[]
        sent=[]
        sentence=str(sentence)
        sentence_1=fuzzy(sentence)
        sentence_words= nlp(sentence_1)
        l=[]
        for i in sentence_words:
             i=str(i)
             l.append(i)
      
        sentence_2.extend(l)
        # lemmatizing the words
        sent=[nlp(word.lower())[0].lemma_ for word in sentence_2]
        sent=[ w  for w in sent if w not in all_stopwords]
        return sent
def bow(sentence ,words):

    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    print(res)
    print("above is res in predict class")
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    print(results)
    print("print above  results")
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print(return_list)
    print("print above return_list")    
    return return_list    

In [383]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            print(result)
            print("above is result in get response")
            break
    return result

def chatbot_response(text):

    fuzzy_1=fuzzy(text)
    print(fuzzy_1,"it is from fuzzy")
    
    ints = predict_class(text, model)
    print(ints)
    print("chatbot")
    res = getResponse(ints, intents)
    tag = ints[0]['intent']
    
    if tag=="order_food":
         k=print_menu()
         n=len(k)
         for i in range(n):
             res=res+" \n "+ k[i]   
    elif tag=="Non-Veg Pizza":
         q=print_nonveg()
         n=len(q)
         for i in range(n):
             res=res+" \n "+ q[i]  

    elif tag=="Veg Pizza":
         v=print_veg()
         n=len(v)
         for i in range(n):
             res=res+" \n "+ v[i]  
    elif tag=="Beverages":
         B=print_bever()
         n=len(B)
         for i in range(n):
             res=res+" \n "+ B[i]  
    elif tag=="Slides":
         s=print_slides()
         n=len(s)
         for i in range(n):
             res=res+" \n "+ s[i]  
    elif tag=="food_item":
         f=cart(fuzzy_1,tag) 
         a,b=f
         items_append.extend(b)
         price_append.extend(a)
         print(price_append)
         total_items(price_append,items_append)
         if (rem_value>0):
             items_append.remove(fuzzy_1)
             res=res+"\n"+"item is removed from the cart"
    elif tag=="confirm_order":
         con= total_items(price_append,items_append)
         n=len(con)
         for i in range(n):
             res=res+" \n "+ con[i]   
    elif tag=="add_menu":
         add=print_menu()
         n=len(add)
         for i in range(n):
             res=res+" \n "+ add[i]  
    elif tag=="remove_menu":
         rem_value=0
         rem_value=rem_value+1
         rem= total_items(price_append,items_append)
         n=len(rem)
         for i in range(n):
             res=res+" \n "+ rem[i]  

    
    return res

In [361]:
def print_nonveg():
    food=[]
    for intent in intents_1['intents']:
        if intent['tag']=="Non-Veg Pizza":
          for i in intent['items']:
            food.append(i)        
    return food       

In [362]:
#method to print the menu
def print_menu():
  l=[]
  for intent in intents_1['intents']:
      l.append(intent['tag'])
  return l

In [363]:
def print_veg():
    food=[]
    for intent in intents_1['intents']:
        if intent['tag']=="Veg Pizza":
            for i in intent['items']:
                food.append(i)        
    return food
print_veg()     

['peppy paneer',
 'Mexican green wave',
 'Veggie paradise',
 'veg extravaganza',
 'deluxe veggie',
 'paneer Makhani',
 'creamy tomato pasta ']

In [364]:
def print_bever():
    food=[]
    for intent in intents_1['intents']:
        if intent['tag']=="Beverages":
            for i in intent['items']:
                food.append(i)        
    return food
print_bever()    

['pepsi', 'slice', '7up', 'mirinda', 'nimbooz']

In [365]:
def print_slides():
    food=[]
    for intent in intents_1['intents']:
        if intent['tag']=="Slides":
            for i in intent['items']:
                food.append(i)        
    return food
print_slides()    

['garlic breadsticks', 'taco mexicana veg', 'Butterscotch mouse cake']

In [ ]:
list_items=[]
for intent in intents['intents']:
        if intent['tag']=="food_item":
            for i in intent['patterns']:
                list_items.append(i)
                      
list_items

In [367]:

def cart(res,tag):
    cart_1=[]
    price=[]
    str2match=res
    strOptions=list_items
    Ratios=process.extract(str2match,strOptions)
    highest=process.extractOne(str2match,strOptions)
    item_name=highest[0]
    cart_1.append(item_name)
    print(cart_1)
    print("the above is cart")
    for intent in intents_1['intents']:
       if intent["tag"]=="Non-Veg Pizza" and item_name in intent["items"]:
               for i in intent['price']:
                   price.append(i)
       elif intent["tag"]=="Veg Pizza" and item_name in intent["items"]:
               for i in intent['price']:
                   price.append(i)
       elif intent["tag"]=="Beverages" and item_name in intent["items"]:
               for i in intent['price']:
                   price.append(i)
       elif intent["tag"]=="Slides" and item_name in intent["items"]:
               for i in intent['price']:
                   price.append(i)          
    print(price,cart_1)                                                       
    return (price,cart_1)   



In [368]:
def total_price(a):
    sum=0
    for i in a:
        sum=sum+i
    total=sum    
    return total

In [372]:
def total_items(a,b):
    total_list=[]
    total_list.extend(b)
    total_value=total_price(a)
    print("Total no. of items in your cart: ")
    for i in total_list:
        print(i)
    print("Total price:")
    print(total_value) 
    return total_list   

In [384]:
k=chatbot_response("pepsi")
print("the belove is k")
print(k)

pepsi
above is fuzzy wuzzy
pepsi it is from fuzzy
pepsi
above is fuzzy wuzzy
[1.6425480e-06 6.6554003e-07 7.7570895e-08 2.2722730e-07 5.5264391e-06
 1.3631727e-06 1.2067186e-06 4.0738774e-07 1.0521975e-06 9.9998343e-01
 2.8722278e-07 1.4277565e-06 2.4535868e-06 2.7218198e-07]
above is res in predict class
[[9, 0.99998343]]
print above  results
[{'intent': 'food_item', 'probability': '0.99998343'}]
print above return_list
[{'intent': 'food_item', 'probability': '0.99998343'}]
chatbot
added to cart
above is result in get response
['pepsi']
the above is cart
[50] ['pepsi']
[50, 50]
Total no. of items in your cart: 
pepsi
pepsi
Total price:
100


UnboundLocalError: ignored